<a href="https://colab.research.google.com/github/nachoacev/practice-data-science/blob/main/FeatureEngineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Engineering

In this course we'll learn about one of the most important steps on the way to building a great machine learning model: feature engineering. We'll learn how to:

- determine which features are the most important with *mutual information*.
- invent new features in several real-world problem domains.
- encode high-cardinality categoricals with a *target encoding*.
- create segmentation features with *k-means clustering*.
- decompose a dataset's variation into features with *principal component analysis*.

The hands-on exercises build up to a complete notebook that applies all of these techniques to make a submission to the House Prices Getting Started competition. After completing this course, you'll have several ideas that you can use to further improve your performance.

## The Goal of Feature Enggineering

The goal of feature engineering is simply *to make your data better suited to the problem at hand.*

Consider "apparent temperature" measures like the heat index and the wind chill. These quantities attempt to measure the perceived temperature to humans based on air temperature, humidity, and wind speed, things which we can measure directly. You could think of an apparent temperature as the result of a kind of feature engineering, an attempt to make the observed data more relevant to what we actually care about: how it actually feels outside!

You might perform feature engineering to:

- improve a model's predictive performance
- reduce computational or data needs
- improve interpretability of the results

For a feature to be useful, *it must have a relationship to the target that your model is able to learn*. Linear models, for instance, are only able to learn linear relationships. So, when using a linear model, your goal is to transform the features to make their relationship to the target linear.

The key idea here is that a transformation you apply to a feature becomes in essence a part of the model itself. Say you were trying to predict the Price of square plots of land from the Length of one side. Fitting a linear model directly to Length gives poor results: the relationship is not linear.